# Sources
* Sequence Encoder model architecture based on [Heigold et al., 2016](https://arxiv.org/abs/1606.06640)
* BiLSTM model architecture based on [Ozols et. al., 2021](https://www.mdpi.com/1422-0067/22/6/3071/htm)

In [1]:
import os
import sys
import csv
import math
import random
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 0 at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m)


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def save_metrics(*args, path):
    if not os.path.isfile(path):
        with open(path, "w", newline="\n") as f:
            f.write(
                ",".join(
                    [
                        "fold",
                        "epoch",
                        "train_loss",
                        "train_acc",
                        "train_auc",
                        "val_loss",
                        "val_acc",
                        "val_auc",
                    ]
                )
            )
            f.write("\n")
    if args:
        with open(path, "a", newline="\n") as f:
            f.write(",".join([str(arg) for arg in args]))
            f.write("\n")


def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        bw_seq = torch.tensor([encode_text(seq)[::-1] for seq in ordered_batch[0]], dtype=torch.int64)
        fw_seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.bw_seq = apply_random_masking(bw_seq, num_tokens=1)
        self.fw_seq = apply_random_masking(fw_seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.bw_seq = self.bw_seq.pin_memory()
        self.fw_seq = self.fw_seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def train_wrapper(batch):
    return TrainBatch(batch)
    
    
class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.bw_seq = torch.tensor([encode_text(seq)[::-1] for seq in ordered_batch[0]], dtype=torch.int64)
        self.fw_seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.bw_seq = self.bw_seq.pin_memory()
        self.fw_seq = self.fw_seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class SeqEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size, dropout):
        super().__init__()
        
        self.fw_embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim
        )
        
        self.bw_embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim
        )
        
        self.dropout = nn.Dropout(dropout)
        
        self.fw_lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=rnn_size, batch_first=True
        )
        
        self.bw_lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=rnn_size, batch_first=True
        )
        
    def forward(self, bw_seq, fw_seq):
        # input shape: (batch_size, seq_len=10)
        fw_embeddings = self.dropout(self.fw_embedding(fw_seq))
        bw_embeddings = self.dropout(self.bw_embedding(bw_seq))
    
        # input shape: (batch_size, seq_len, embedding_dim)
        fw_out, _ = self.fw_lstm(fw_embeddings)
        bw_out, _ = self.bw_lstm(bw_embeddings)
        
        # input shape: (batch_size, seq_len, rnn_size)
        # only get representation at last t
        fw_out = self.dropout(fw_out[:, -1, :])
        bw_out = self.dropout(bw_out[:, -1, :])
        
        # input shape: (batch_size, rnn_size)
        # out shape: (batch_size, 2*rnn_size)
        return torch.cat([fw_out, bw_out], dim=1) 
        
    

class FwBwQuadBiLSTM(nn.Module):
    def __init__(self, vocab_size, seq_enc_emb_dim, seq_enc_rnn_size, rnn_size1, rnn_size2, hidden_size, dropout):
        super().__init__()
        
        # sequence encoder replaces embedding representations
        self.seq_encoder = SeqEncoder(
            vocab_size=vocab_size,
            embedding_dim=seq_enc_emb_dim,
            rnn_size=seq_enc_rnn_size,
            dropout=dropout
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=seq_enc_rnn_size * 2,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm2 = nn.LSTM(
            input_size=2*rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True
        )
        
        self.fc1 = nn.Linear(rnn_size2 * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, bw_seq, fw_seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.seq_encoder(bw_seq, fw_seq))
        
        # input shape: (batch_size, seq_enc_rnn_size * 2)
        out, _ = self.lstm1(embedded)
        
        # input shape: (batch_size, 2*rnn_size)
        out, _ = self.lstm2(out)
        
        # input shape; (batch_size, 2*rnn_size)
        out = self.dropout(gelu(self.fc1(out)))
        
        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [6]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        bw_seq, fw_seq, lbl = batch.bw_seq, batch.fw_seq, batch.lbl
        bw_seq, fw_seq, lbl = bw_seq.to(device), fw_seq.to(device), lbl.to(device)
        
        scores = model(bw_seq, fw_seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += bw_seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [8]:
NUM_EPOCHS = 15
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
SEQ_ENC_EMB_DIM = 100
SEQ_ENC_RNN_SIZE = 200
RNN_SIZE1 = 128
RNN_SIZE2 = 512
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 1e-4

model = FwBwQuadBiLSTM(
    vocab_size=VOCAB_SIZE,
    seq_enc_emb_dim=SEQ_ENC_EMB_DIM,
    seq_enc_rnn_size=SEQ_ENC_RNN_SIZE,
    rnn_size1=RNN_SIZE1,
    rnn_size2=RNN_SIZE2,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

In [9]:
start = time()
print("Starting Training.")
logging_path = "../../params/n_term/FwBwBiLSTM/results.csv"
highest_val_auc = 0
fold=None

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc, train_auc = process(model, train_loader, criterion, optimizer)

    model.eval()
    with torch.no_grad():
        val_loss, val_acc, val_auc = process(model, dev_loader, criterion)

    # save metrics
    save_metrics(
        fold,
        epoch,
        train_loss,
        train_acc,
        train_auc,
        val_loss,
        val_acc,
        val_auc,
        path=logging_path,
    )

    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.6f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.6f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")

    reg_auc = regularized_auc(train_auc, val_auc, threshold=0)
    if reg_auc > highest_val_auc:
        highest_val_auc = reg_auc
        path = f"../../params/n_term/FwBwBiLSTM/auc{reg_auc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)

print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|███████████████████████████████████████████████████████| 281/281 [00:04<00:00, 64.27batches/s]
Training:   [Epoch  1, Loss: 0.000900, Acc: 0.8166, AUC: 0.6175]
Evaluation: [Epoch  1, Loss: 0.000806, Acc: 0.8227, AUC: 0.7548]
Eval: 100%|███████████████████████████████████████████████████████| 281/281 [00:04<00:00, 61.79batches/s]
Training:   [Epoch  2, Loss: 0.000833, Acc: 0.8199, AUC: 0.7230]
Evaluation: [Epoch  2, Loss: 0.000794, Acc: 0.8271, AUC: 0.7650]
Eval: 100%|███████████████████████████████████████████████████████| 281/281 [00:04<00:00, 61.93batches/s]
Training:   [Epoch  3, Loss: 0.000822, Acc: 0.8213, AUC: 0.7344]
Evaluation: [Epoch  3, Loss: 0.000790, Acc: 0.8280, AUC: 0.7698]
Eval: 100%|███████████████████████████████████████████████████████| 281/281 [00:04<00:00, 63.43batches/s]
Training:   [Epoch  4, Loss: 0.000815, Acc: 0.8226, AUC: 0.7412]
Evaluation: [Epoch  4, Loss: 0.000786, Acc: 0.8266, AUC: 0.7730]
Eval: 100%|██████████████████████████

In [12]:
test_path = '../../data/n_test.csv'
test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/n_term/FwBwBiLSTM/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model.load_state_dict(torch.load('../../params/n_term/FwBwBiLSTM/' + best_model))
model.eval()
test_loss, test_acc, test_auc = process(model, test_loader, criterion)
print(
    f"Test Set Performance: Loss: {test_loss:.6f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)
print(
    f"Total model params: {total_model_params(model)}, trainable model params: {trainable_model_params(model)}"
)

Loaded model:  auc0.7847_epoch15.pt
Eval: 100%|███████████████████████████████████████████████████████| 281/281 [00:04<00:00, 59.95batches/s]
Test Set Performance: Loss: 0.000764, Acc: 0.8315, AUC: 0.7871
Total model params: 4315369, trainable model params: 4315369
